In [1]:
# Importing necessary libraries
import praw
import pandas as pd
import os
import time
from datetime import datetime, timezone
from tqdm import tqdm

In [2]:
# reddit api credentials
REDDIT_CLIENT_ID = "0ksEDSoxha6QYCo63d42gQ"
REDDIT_CLIENT_SECRET = "hgT8R03-G6RVh7nY7vhXa_YHRQHGNw"
REDDIT_USER_AGENT = "bgood8888"

In [3]:
# Data Directory
DATA_DIR = "Data"
os.makedirs(DATA_DIR, exist_ok=True)

In [4]:
# Subreddits to scrape
SUBREDDITS = [
    'suicidewatch', 'depression', 'ptsd', 'anxiety',
    'socialanxiety', 'healthanxiety', 'bipolarreddit',
    'bpd', 'schizophrenia', 'paranoia', 'EDAnonymous', 'alcoholism', 'addiction', 'adhd', 'mindfulness',
    'psychosis', 'mentalillness', 'mentalhealth', 'meditation', 'personalfinance', 'jokes', 'teaching',
    'relationships', 'legaladvice', 'fitness', 'parenting', 'COVID19_support', 'lonely', 'autism', 'divorce',
    'guns', 'economy', 'ForeverAlone', 'conspiracy', 'MakeNewFriendsHere', 'AskReddit', 'offmychest', 'LifeProTips',
    'TrueOffMyChest', 'unpopularopinion', 'CasualConversation', 'Advice', 'selfimprovement', 'nosleep', 'todayilearned',
    'AskMen', 'AskWomen', 'AntiMLM', 'changemyview', 'confession', 'decidingtobebetter', 'mentalhealthUK', 'support',
    'stopdrinking', 'getdisciplined', 'tifu', 'careerguidance', 'psychology', 'Wellthatsucks', 'selfcare', 'Stress',
    'socialskills', 'Therapy', 'friendship', 'caregivers', 'Loneliness', 'sad', 'BPDlovedones', 'aspergers', 'autismparents'
]

In [5]:
# Initialize Reddit instance
reddit = praw.Reddit(
    client_id = REDDIT_CLIENT_ID,
    client_secret = REDDIT_CLIENT_SECRET,
    user_agent = REDDIT_USER_AGENT
)

In [6]:
def clean_text(txt):
    if isinstance(txt, str):
        return txt.replace('\n', ' ').replace('\r', ' ').strip()
    return ''

def get_scraped_ids(filepath):
    """Return set of post_ids already saved in CSV, if file exists."""
    if os.path.exists(filepath):
        try:
            df = pd.read_csv(filepath, usecols=["post_id"])
            return set(df["post_id"].astype(str))
        except Exception:
            return set()
    return set()

def scrape_subreddit(subreddit, post_limit=None, comment_limit=50):
    posts_csv = os.path.join(DATA_DIR, f"{subreddit}_posts.csv")
    comments_csv = os.path.join(DATA_DIR, f"{subreddit}_comments.csv")
    scraped_ids = get_scraped_ids(posts_csv)
    posts = []
    comments = []
    new_posts = 0

    try:
        subreddit_obj = reddit.subreddit(subreddit)
        for submission in tqdm(subreddit_obj.new(limit=post_limit), desc=f"r/{subreddit} posts"):
            if submission.id in scraped_ids:
                continue
            post_id = submission.id
            # FIXED: Use timezone-aware UTC datetime
            post_data = {
                'post_id': post_id,
                'content': clean_text(submission.title) + ' ' + clean_text(submission.selftext),
                'author': str(submission.author) if submission.author else '',
                'date': datetime.fromtimestamp(submission.created_utc, timezone.utc).isoformat() if hasattr(submission, 'created_utc') else '',
                'category': subreddit,
                'num_comments': submission.num_comments,
                'score': submission.score,
                'upvote_ratio': submission.upvote_ratio,
                'url': submission.url
            }
            posts.append(post_data)
            new_posts += 1

            # Fetch comments (up to comment_limit)
            try:
                submission.comments.replace_more(limit=0)
                comment_list = submission.comments.list()
                comment_list = sorted(comment_list, key=lambda x: getattr(x, 'created_utc', 0))
                for idx, c in enumerate(comment_list[:comment_limit]):
                    comments.append({
                        'comment_id': getattr(c, 'id', ''),
                        'post_id': post_id,
                        'content': clean_text(getattr(c, 'body', '')),
                        'author': str(getattr(c, 'author', '')) if getattr(c, 'author', None) else '',
                        'date': datetime.fromtimestamp(getattr(c, 'created_utc', 0), timezone.utc).isoformat() if hasattr(c, 'created_utc') else '',
                        'order': idx + 1,
                        'score': getattr(c, 'score', ''),
                        'parent_id': getattr(c, 'parent_id', ''),
                        'link_id': getattr(c, 'link_id', ''),
                        'permalink': getattr(c, 'permalink', '')
                    })
            except Exception as ce:
                print(f"Error fetching comments for post {post_id} in r/{subreddit}: {ce}")

            # Save progress every 100 posts
            if new_posts % 100 == 0:
                if posts:
                    pd.DataFrame(posts).to_csv(posts_csv, mode='a', header=not os.path.exists(posts_csv), index=False)
                    posts = []
                if comments:
                    pd.DataFrame(comments).to_csv(comments_csv, mode='a', header=not os.path.exists(comments_csv), index=False)
                    comments = []
                time.sleep(2)  # brief pause

    except Exception as e:
        print(f"Error scraping r/{subreddit}: {e}")

    # Save any remaining data
    if posts:
        pd.DataFrame(posts).to_csv(posts_csv, mode='a', header=not os.path.exists(posts_csv), index=False)
    if comments:
        pd.DataFrame(comments).to_csv(comments_csv, mode='a', header=not os.path.exists(comments_csv), index=False)
    print(f"Saved {new_posts} new posts for r/{subreddit}.")
    time.sleep(30)  # pause between subreddits

# Main loop
for sub in tqdm(SUBREDDITS, desc="Subreddits"):
    scrape_subreddit(sub, post_limit=None, comment_limit=50)  # None = as many as Reddit allows

Subreddits:   0%|                                                                               | 0/66 [00:00<?, ?it/s]
r/socialanxiety posts: 0it [00:00, ?it/s]
r/socialanxiety posts: 1it [00:02,  2.73s/it]
r/socialanxiety posts: 2it [00:03,  1.36s/it]
r/socialanxiety posts: 3it [00:03,  1.11it/s]
r/socialanxiety posts: 4it [00:03,  1.39it/s]
r/socialanxiety posts: 5it [00:04,  1.71it/s]
r/socialanxiety posts: 6it [00:04,  1.90it/s]
r/socialanxiety posts: 7it [00:05,  2.13it/s]
r/socialanxiety posts: 8it [00:05,  2.25it/s]
r/socialanxiety posts: 9it [00:05,  2.34it/s]
r/socialanxiety posts: 10it [00:06,  2.44it/s]
r/socialanxiety posts: 11it [00:06,  2.37it/s]
r/socialanxiety posts: 12it [00:08,  1.36it/s]
r/socialanxiety posts: 13it [00:09,  1.14it/s]
r/socialanxiety posts: 14it [00:09,  1.34it/s]
r/socialanxiety posts: 15it [00:10,  1.58it/s]
r/socialanxiety posts: 16it [00:10,  1.83it/s]
r/socialanxiety posts: 17it [00:10,  1.97it/s]
r/socialanxiety posts: 18it [00:11,  2.18it/s]
r

Saved 970 new posts for r/socialanxiety.


Subreddits:   2%|█                                                                  | 1/66 [09:27<10:14:33, 567.28s/it]
r/healthanxiety posts: 0it [00:00, ?it/s]
r/healthanxiety posts: 1it [00:02,  2.02s/it]
r/healthanxiety posts: 2it [00:02,  1.04s/it]
r/healthanxiety posts: 3it [00:02,  1.33it/s]
r/healthanxiety posts: 4it [00:03,  1.63it/s]
r/healthanxiety posts: 5it [00:03,  1.66it/s]
r/healthanxiety posts: 6it [00:04,  1.74it/s]
r/healthanxiety posts: 7it [00:04,  1.70it/s]
r/healthanxiety posts: 8it [00:05,  1.93it/s]
r/healthanxiety posts: 9it [00:05,  2.09it/s]
r/healthanxiety posts: 10it [00:06,  2.21it/s]
r/healthanxiety posts: 11it [00:06,  2.31it/s]
r/healthanxiety posts: 12it [00:06,  2.27it/s]
r/healthanxiety posts: 13it [00:07,  2.23it/s]
r/healthanxiety posts: 14it [00:07,  2.29it/s]
r/healthanxiety posts: 15it [00:08,  2.31it/s]
r/healthanxiety posts: 16it [00:09,  1.79it/s]
r/healthanxiety posts: 17it [00:09,  1.99it/s]
r/healthanxiety posts: 18it [00:09,  1.98it/s]
r

Saved 981 new posts for r/healthanxiety.


Subreddits:   3%|██                                                                 | 2/66 [22:52<12:34:40, 707.51s/it]
r/bipolarreddit posts: 0it [00:00, ?it/s]
r/bipolarreddit posts: 1it [00:01,  1.95s/it]
r/bipolarreddit posts: 2it [00:02,  1.00s/it]
r/bipolarreddit posts: 3it [00:02,  1.39it/s]
r/bipolarreddit posts: 4it [00:03,  1.72it/s]
r/bipolarreddit posts: 5it [00:03,  1.99it/s]
r/bipolarreddit posts: 6it [00:03,  2.19it/s]
r/bipolarreddit posts: 7it [00:04,  2.36it/s]
r/bipolarreddit posts: 8it [00:04,  2.42it/s]
r/bipolarreddit posts: 9it [00:04,  2.41it/s]
r/bipolarreddit posts: 10it [00:05,  2.19it/s]
r/bipolarreddit posts: 11it [00:05,  2.33it/s]
r/bipolarreddit posts: 12it [00:06,  2.14it/s]
r/bipolarreddit posts: 13it [00:06,  2.19it/s]
r/bipolarreddit posts: 14it [00:07,  2.22it/s]
r/bipolarreddit posts: 15it [00:07,  2.27it/s]
r/bipolarreddit posts: 16it [00:08,  2.26it/s]
r/bipolarreddit posts: 17it [00:08,  2.30it/s]
r/bipolarreddit posts: 18it [00:08,  2.36it/s]
r

...

Subreddits:  97%|█████████████████████████████████████████████████████████████████▉  | 64/66 [9:08:01<02:44, 82.43s/it]
r/aspergers posts: 0it [00:51, ?it/s]


Error scraping r/aspergers: error with request HTTPSConnectionPool(host='oauth.reddit.com', port=443): Read timed out. (read timeout=16.0)
Saved 0 new posts for r/aspergers.


Subreddits:  98%|██████████████████████████████████████████████████████████████████▉ | 65/66 [9:09:23<01:22, 82.29s/it]
r/autismparents posts: 0it [00:53, ?it/s]


Error scraping r/autismparents: error with request HTTPSConnectionPool(host='oauth.reddit.com', port=443): Read timed out. (read timeout=16.0)
Saved 0 new posts for r/autismparents.


Subreddits: 100%|███████████████████████████████████████████████████████████████████| 66/66 [9:10:47<00:00, 500.72s/it]
